In [1]:
%autosave 60 

Autosaving every 60 seconds


In [2]:
#default_exp segmentation.losses

In [3]:
import fastai; print(fastai.__version__)

1.0.58.dev0


In [4]:
#export
from fastai.vision import *
from local.segmentation.lovasz_loss import *

In [5]:
#export
_all_ = ["lovasz_hinge", "lovasz_softmax"]

In [6]:
# export
cross_entropy = CrossEntropyFlat(axis=1)

In [123]:
#export
def dice_loss(input, target):
    "input: logits (B,2,H,W), target: labels (B,1,H,W)"
    smooth = 1.
    input = input.softmax(1)[:,1,:].unsqueeze(1)
    iflat = input.view(-1)
    tflat = target.view(-1).float()
    intersection = (iflat * tflat).sum()
    return 1 - ((2. * intersection + smooth) / (iflat.sum() + tflat.sum() + smooth))

### test

In [8]:
from local.test import *

### `cross_entropy`

In [9]:
_input = torch.ones((1,2,3,3))

_target = tensor([[[
    [0,0,0],
    [0,1,1],
    [0,1,1]
]]])

_input.size(), _target.size()

(torch.Size([1, 2, 3, 3]), torch.Size([1, 1, 3, 3]))

In [10]:
test_close(cross_entropy(_input, _target), tensor(0.6931), eps=1e-4)

###  `lovasz_hinge` -> (best:0, random:1, worst:inf)

https://www.kaggle.com/c/tgs-salt-identification-challenge/discussion/67791

In [109]:
_input = torch.zeros((1,3,3))

_input2 = tensor([[
    [-1e20,-1e20,-1e20],
    [-1e20,1e20,1e20],
    [-1e20,1e20,1e20]
]]).float()

_input3 = tensor([[
    [1e20,1e20,1e20],
    [1e20,-1e20,-1e20],
    [1e20,-1e20,-1e20]
]]).float()

_target = tensor([[
    [0,0,0],
    [0,1,1],
    [0,1,1]
]])

_input.size(), _target.size()

(torch.Size([1, 3, 3]), torch.Size([1, 3, 3]))

In [110]:
# random
lovasz_hinge(_input, _target)

tensor(1.)

In [111]:
# best
lovasz_hinge(_input2, _target)

tensor(0.)

In [112]:
# worst
lovasz_hinge(_input3, _target)

tensor(1.0000e+20)

### `lovasz_softmax` -> (best:0, random:0.5, worst:1)

In [66]:
_input = torch.ones((1,2,3,3))

_input2 = tensor([[
[
    [1,1,1],
    [1,0,0],
    [1,0,0]
],
[
    [0,0,0],
    [0,1,1],
    [0,1,1]
]
]]).float()*1e3

_input3 = tensor([[
[
    [1,1,1],
    [1,0,0],
    [1,0,0]
],
[
    [0,0,0],
    [0,1,1],
    [0,1,1]
]
]]).float()*-1e3

_target = tensor([[[
    [0,0,0],
    [0,1,1],
    [0,1,1]
]]])

_input.size(), _target.size()

(torch.Size([1, 2, 3, 3]), torch.Size([1, 1, 3, 3]))

In [71]:
# random
(lovasz_softmax(_input, _target))

tensor(0.5000)

In [72]:
# best
(lovasz_softmax(_input2, _target))

tensor(0.)

In [73]:
# worst
(lovasz_softmax(_input3, _target))

tensor(1.)

### `dice_loss`

In [138]:
_input = torch.ones((1,2,3,3))

_input2 = tensor([[
[
    [1,1,1],
    [1,0,0],
    [1,0,0]
],
[
    [0,0,0],
    [0,1,1],
    [0,1,1]
]
]]).float()*1e3

_input3 = tensor([[
[
    [1,1,1],
    [1,0,0],
    [1,0,0]
],
[
    [0,0,0],
    [0,1,1],
    [0,1,1]
]
]]).float()*-1e3

_target = tensor([[[
    [0,0,0],
    [0,1,1],
    [0,1,1]
]]])

_input.size(), _target.size()

(torch.Size([1, 2, 3, 3]), torch.Size([1, 1, 3, 3]))

In [142]:
# random
dice_loss(_input, _target)

tensor(0.4737)

In [143]:
# best
dice_loss(_input2, _target)

tensor(0.)

In [144]:
# worst
dice_loss(_input3, _target)

tensor(0.9000)

### export

In [146]:
from local.notebook.export import notebook2script
notebook2script(all_fs=True)

Converted 00_test.ipynb.
Converted 01_script.ipynb.
Converted 02_scheduler.ipynb.
Converted 03_callbacks.ipynb.
Converted 10_segmentation_dataset.ipynb.
Converted 11_segmentation_losses.ipynb.
Converted 12_segmentation_metrics.ipynb.
Converted 13_segmentation_models.ipynb.
Converted segmentation_training.ipynb.
